In [69]:
import pandas as pd
import numpy as np
from datetime import datetime

In [70]:
data_path='TestPort.csv'
trade_plan_path='ListTradeTran.xlsx'
asset_comm_path='AssetInfo.xlsx'
result_path='Report_TestPort_20T22.xlsx'

In [71]:
TotalBudget=100000
# df=pd.read_csv('ASP-SME_KKP-SM-TRade.csv',index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df=pd.read_csv(data_path,parse_dates=['Date/Time'],dayfirst=True)
# df.index.set_names('date',inplace=True)
df.columns=['symbol','date','price']

list_symbols=df['symbol'].unique()
print(list_symbols)
df.info()
print(df.tail())

['ASP-SME' 'ASP-THEQ' 'KKP-ACT-EQ-A' 'KKP-SM-CAP' 'TISCOMS-A' 'TISCOFLEXP'
 'KFDYNAMIC' 'TSF-A']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5826 entries, 0 to 5825
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   symbol  5826 non-null   object        
 1   date    5826 non-null   datetime64[ns]
 2   price   5826 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 136.7+ KB
     symbol       date  price
5821  TSF-A 2022-12-30  84.53
5822  TSF-A 2023-01-03  85.13
5823  TSF-A 2023-01-04  83.97
5824  TSF-A 2023-01-05  83.92
5825  TSF-A 2023-01-06  84.42


In [72]:
dfPlan=pd.read_excel(trade_plan_path,engine="openpyxl",sheet_name='TradePlan', \
                     parse_dates=['trade_date'],dtype ={'portion_pct': np.float64})
dfAssetInfo=pd.read_excel(asset_comm_path,engine="openpyxl",sheet_name='AssetInfo',\
                    index_col='symbol', dtype ={'buy_comm_pct': np.float64,'sell_comm_pct': np.float64})

tradeDateList= [ d for d in list(dfPlan['trade_date']) ]

print(tradeDateList)
print("================================================================================================")

print(dfPlan)
print(dfAssetInfo)


[Timestamp('2020-12-01 00:00:00'), Timestamp('2021-01-05 00:00:00'), Timestamp('2021-01-15 00:00:00'), Timestamp('2021-02-01 00:00:00'), Timestamp('2022-01-19 00:00:00'), Timestamp('2022-02-10 00:00:00'), Timestamp('2022-03-03 00:00:00'), Timestamp('2022-04-29 00:00:00')]
  trade_date  portion_pct action
0 2020-12-01         30.0      b
1 2021-01-05         20.0      b
2 2021-01-15         20.0      b
3 2021-02-01         30.0      b
4 2022-01-19         40.0      s
5 2022-02-10         20.0      s
6 2022-03-03         20.0      s
7 2022-04-29         20.0      s
              buy_comm_pct  sell_comm_pct
symbol                                   
ASP-SME               1.25            0.0
ASP-THEQ              1.00            0.0
KKP-SM-CAP            1.00            0.0
TISCOFLEXP            1.00            0.0
TSF-A                 1.05            0.0
TISCOMS-A             1.00            0.0
KFDYNAMIC             0.50            0.5
KKP-ACT-EQ-A          0.50            0.0


In [73]:
# veridf  there are both buy and sell

# verify b and sell , sum of each action have to be 100  

# veriry all list in data price are in commision

In [74]:
def set_commision_by_action(item):
  x=item['symbol']  
  if item['action']=='b':
    comm_type='buy_comm_pct'
  elif  item['action']=='s':
    comm_type='sell_comm_pct'
  try:
     return dfAssetInfo.loc[x,comm_type]
  except Exception as ex:
     print(f"Error : no {x} to get commission")
     return 0
def get_qty_buy(item):
    xCost=(item['portion_pct']/100)*TotalBudget
    xPrice=item['price_on_comm']
    qty=xCost/xPrice
    return qty
def get_qty_sell(item,TotalQty):
    qty=(item['portion_pct']/100)*TotalQty
    return qty

In [76]:
dict_dfTran={}
dfSummary=pd.DataFrame(columns=['Symbol','BuyValue','AvgBuyPrice','SellValue','AvgSellPrice','PL','Percent_PL'])
for symbol in list_symbols:
    print(symbol)
    dfBS=df[ (df['symbol']==symbol) & (df['date'].isin(tradeDateList))]
    dfBS=dfBS.merge(dfPlan,how='inner',left_on='date',right_on='trade_date')
    
    dfBS['comm']=dfBS.apply(set_commision_by_action,axis=1)
    dfBS['price_on_comm']=dfBS.apply(lambda item : item['price']*(1+ (item['comm']/100)) if item['action']=='b' else item['price']*(1-(item['comm']/100)), axis=1)
    dfBS['price_on_comm']=dfBS['price_on_comm'].round(2)
    
    dfBS=dfBS.drop(columns=['trade_date'])
    #print(dfBS)
    
    print("================Buy=================")
    buyDF=dfBS.query("action=='b'")    
    # cal qury and actual value and  total_val and totl qury
    buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
    buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
    buyDF['action_value']=buyDF['value']
    print(buyDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    TotalQty=buyDF['qty'].sum()
    TotalValue=round(buyDF['value'].sum(),2)
    AvgPrice=round(TotalValue/TotalQty,2)
    # print(f"Total Buy {symbol} : qty={TotalQty} and value={TotalValue} at AvgPrice={AvgPrice}")
    
    print("=================Sell=================")
    sellDF=dfBS.query("action=='s'")  
    sellDF['qty']=sellDF.apply(get_qty_sell,axis=1,args=(TotalQty,))
    sellDF['value']=sellDF['qty']*sellDF['price_on_comm']
    sellDF['action_value']=sellDF['value']*-1
    print(sellDF[['date' ,'price_on_comm','qty','value','portion_pct']])
    
    SellTotalQty=sellDF['qty'].sum()
    SellTotalValue=round(sellDF['value'].sum(),2)
    SellAvgPrice=round(SellTotalValue/SellTotalQty,2)
    
    # print(f"Total Sell {symbol} : qty={SellTotalQty} and value={SellTotalValue} at AvgPrice={SellAvgPrice}")
    
    PL=round(SellTotalValue-TotalValue,0)
    PCT_PL=round(PL/TotalValue*100,2)
    
    summary_data={'Symbol':symbol,'BuyValue':TotalValue,'AvgBuyPrice':AvgPrice,'SellValue':SellTotalValue,'AvgSellPrice':SellAvgPrice, \
                  'PL':PL,'Percent_PL':PCT_PL
                 }
    dfSummary=dfSummary.append(summary_data,ignore_index=True)
    
    dfTran=pd.concat([buyDF,sellDF],axis=0)
    dfTran=dfTran.sort_values(by=['date'])
    dfTran[['value','action_value']]=dfTran[['value','action_value']].round(0)
    
    dict_dfTran[symbol]=dfTran
    
    
    
    

    


ASP-SME
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2020-12-01          12.46  2407.704655  30000.0         30.0
1 2021-01-05          13.31  1502.629602  20000.0         20.0
2 2021-01-15          13.57  1473.839352  20000.0         20.0
3 2021-02-01          14.01  2141.327623  30000.0         30.0
=================Sell=================
        date  price_on_comm          qty         value  portion_pct
4 2022-01-19          19.08  3010.200493  57434.625398         40.0
5 2022-02-10          18.30  1505.100246  27543.334507         20.0
6 2022-03-03          18.42  1505.100246  27723.946536         20.0
7 2022-04-29          18.91  1505.100246  28461.445657         20.0
ASP-THEQ
================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2020-12-01          11.66  2572.898799  30000.0         30.0
1 2021-01-05          12.25  1632.653061  20000.0         20.0
2 2021-01-15          12.3

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

================Buy=================
        date  price_on_comm          qty    value  portion_pct
0 2020-12-01          16.83  1782.531194  30000.0         30.0
1 2021-01-05          17.36  1152.073733  20000.0         20.0
2 2021-01-15          17.62  1135.073780  20000.0         20.0
3 2021-02-01          18.30  1639.344262  30000.0         30.0
=================Sell=================
        date  price_on_comm          qty         value  portion_pct
4 2022-01-19          22.46  2283.609188  51289.862354         40.0
5 2022-02-10          21.91  1141.804594  25016.938651         20.0
6 2022-03-03          22.11  1141.804594  25245.299569         20.0
7 2022-04-29          22.64  1141.804594  25850.456004         20.0
TISCOFLEXP
================Buy=================
        date  price_on_comm         qty    value  portion_pct
0 2020-12-01          43.17  694.927033  30000.0         30.0
1 2021-01-05          47.14  424.268137  20000.0         20.0
2 2021-01-15          48.67  410.93

C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['qty']=buyDF.apply(get_qty_buy,axis=1)
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  buyDF['value']=buyDF['qty']*buyDF['price_on_comm']
C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\3649325214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [65]:
writer=pd.ExcelWriter(result_path,engine='xlsxwriter') 
dfSummary.to_excel(writer, sheet_name="Summary",index=False)
print(dfSummary)
for symbol,tranDF in dict_dfTran.items():
 tranDF.to_excel(writer, sheet_name=symbol,index=False) 
 #print(tranDF)
writer.save()

writer.close()

print("Successfully")


       Symbol  BuyValue  AvgBuyPrice  SellValue  AvgSellPrice       PL  \
0     ASP-SME  100000.0        13.29  141163.35         18.76  41163.0   
1    ASP-THEQ  100000.0        12.18  127245.64         15.50  27246.0   
2  KKP-SM-CAP  100000.0        13.55  135599.57         18.38  35600.0   
3  TISCOFLEXP  100000.0        46.93  133927.02         62.85  33927.0   
4       TSF-A  100000.0        66.72  125798.95         83.93  25799.0   
5   TISCOMS-A  100000.0        17.52  127402.56         22.32  27403.0   

   Percent_PL  
0       41.16  
1       27.25  
2       35.60  
3       33.93  
4       25.80  
5       27.40  


C:\Users\pongthsa\AppData\Local\Temp\ipykernel_29440\2721000547.py:7: FutureWarning: save is not part of the public API, usage can give in unexpected results and will be removed in a future version
  writer.save()


Successfully


D:\ProgramData\Anaconda3\envs\myquant_exp\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
